# Imports

In [173]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import zipfile
import itertools
from typing import List

%matplotlib inline

keywords = List[str]
Datasets = List[pd.DataFrame]

# Constants

In [2]:
base_url = 'https://drive.google.com/uc?export=download'
survey_ids = {
    2011: '0Bx0LyhBTBZQgUGVYaGx3SzdUQ1U',
    2012: '0B0DL28AqnGsrX3JaZWVwWEpHNWM',
    2013: '0B0DL28AqnGsrenpPNTc5UE1PYW8',
    2014: '0B0DL28AqnGsrempjMktvWFNaQzA',
    2015: '0B0DL28AqnGsra1psanV1MEdxZk0',
    2016: '0B0DL28AqnGsrV0VldnVIT1hyb0E',
    2017: '0B6ZlG_Eygdj-c1kzcmUxN05VUXM',
    2018: '1_9On2-nsBQIw3JiY43sWbrF8EjrqrR4U',
    2019: '1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV'
}


# Functions

In [178]:
def download_data(url: str, file_id: str):
    
    response = requests.get(
        url=url,
        params={
            'id': file_id
        },
        stream=True
    )
    return response

def unzip_file(file_name: str, extract_to_folder: str, file_format: str='CSV'):
    
    zipped_file = zipfile.ZipFile(file_name)
    for item in zipped_file.filelist:
        in_file = item.filename
        if not os.path.exists(os.path.join(str(extract_to_folder), in_file)):
            if in_file.upper().endswith(file_format.upper()):
                zipfile.ZipFile(file_name).extract(in_file, path=str(extract_to_folder))
                

def read_csv_files(file_format: str='CSV', key_words: keywords=['RESULTS']) -> Datasets:
    
    dfs = dict()
    
    for root, dirs, files in os.walk(top='.'):
        [dirs.remove(d) if d in ['__MACOSX', '.git'] else None for d in dirs]
        
        for file in files:
            if file.upper().endswith('CSV') and \
            any(val.upper() in file.upper() for val in key_words):
                try:
                    dfs[root[2:6]] = (pd.read_csv(os.path.join(root, file)))
                except UnicodeDecodeError:
                    print('An error occurred while decoding "{in_file}"')
    return dfs

def get_data(url: str, file_ids: dict, download_files: bool=False, file_format: str='CSV', key_words: keywords=['RESULTS']) -> Datasets:
    
    key_values = []
    
    for survey_year, file_id in file_ids.items():
        
        file_name = os.path.join(str(survey_year), 'data.zip')
        key_values.append(survey_year)
        
        if download_files:
            try:
                response = download_data(url, file_id)
                os.makedirs(str(survey_year))
                with open(file_name, 'wb') as file:
                    file.write(response.content)
            except FileExistsError:
                print(f'File "{file_name}" already exists.')
        
        unzip_file(file_name, survey_year)    
        
    dfs = read_csv_files(key_words=key_words)
    return dfs

In [179]:
dfs = get_data(download_files=False, url=base_url, file_ids=survey_ids, key_words=[' results', 'responses', 'results_public'])

/home/lior/apps/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (5,108,121,196,197,198) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/lior/apps/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (46,48,62) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/lior/apps/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types.Specify dtype option on import or set low_memory=False.
  exec

An error occurred while decoding "{in_file}"
An error occurred while decoding "{in_file}"


In [180]:
dfs.keys()

dict_keys(['2014', '2015', '2019', '2013', '2018', '2016', '2017'])

In [195]:
print(len(dfs['2016'].columns.tolist()))

66
